In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
# 0. 필요 라이브러리 설치
!pip install datasets transformers scikit-learn torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# 1. 라이브러리 임포트
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
import torch
from sklearn.metrics import accuracy_score


In [4]:
# 2. KoBERT 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
print("✅ KoBERT 토크나이저 로드 완료")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

✅ KoBERT 토크나이저 로드 완료


In [5]:
# 3. CSV 파일 → Hugging Face Dataset 로드
dataset = load_dataset("csv", data_files={"train": "review.csv"})
print("✅ CSV 로드 완료")

Generating train split: 0 examples [00:00, ? examples/s]

✅ CSV 로드 완료


1. csv -> dataset으로 변환 dataset 변환해야하는 이유는 아래와 같음.

CSV 파일을 바로 로드해서 학습 파이프라인에 넣을 수도 있지만, 가능하면 Hugging Face의 datasets 라이브러리나 PyTorch Dataset 형식 등 표준화된 Dataset 객체로 변환해서 사용하는 편이 더 좋습니다. 주된 이유는 다음과 같습니다:

전처리 및 관리의 용이성

CSV를 그대로 로드해서 사용할 경우, 매번 로드할 때마다 필요한 전처리(불용어 제거, 토크나이징 등)를 일일이 코드로 구현해야 합니다.
Dataset 객체로 변환해 두면, 전처리 로직(예: 토크나이저 적용)을 체계적으로 관리할 수 있고, 필요하면 캐싱(caching) 기능 등을 활용해 보다 효율적으로 데이터를 다룰 수 있습니다.
데이터 분할(train/validation/test) 자동화

datasets 라이브러리의 load_dataset("csv", ...) 같은 기능을 사용하면, CSV를 쉽게 불러오면서 바로 train/validation/test 세트로 분할하거나 섞는 등의 작업을 간편하게 할 수 있습니다.
커스텀 파이프라인을 짜더라도 Dataset 객체를 상속받아 구현하면 다양한 편의 기능(배치, 샘플링, 셔플 등)을 쉽게 사용할 수 있습니다.
데이터셋 메타정보 관리

CSV 파일 메타정보(열 이름, 데이터 타입 등)를 Dataset 객체로 변환 시 함께 저장해두면, 나중에 어떤 칼럼이 어떤 의미인지 확인하기가 편리합니다.
추가적으로, Dataset 객체는 .map(), .filter(), .shuffle() 등의 메서드를 제공하여, 데이터 처리를 함수형 스타일로 간단히 적용하기 좋습니다.
재사용성과 확장성

여러 프로젝트나 모델(예: KoBERT, 다른 BERT 모델, GPT-계열 모델)에서 동일한 데이터셋을 사용하게 될 때, Dataset 객체를 만들어 두면 재사용하기 훨씬 용이합니다.
추후 다른 전처리나 추가 레이블 작업을 하게 될 경우에도 Dataset 객체를 기반으로 쉽게 작업을 이어갈 수 있습니다.

In [6]:
# 데이터 컬럼명 확인
print("📊 데이터 컬럼:", dataset["train"].column_names)
# 출력 예) ["review", "label"]

📊 데이터 컬럼: ['review', 'label']


In [7]:
# 4. 토크나이저 적용 함수
################################


def tokenize_function(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True, max_length=128, return_token_type_ids=False )


In [8]:
# 5. 전체 데이터셋에 토큰화 적용

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 원본 컬럼 "review" 삭제 (이미 토큰으로 변환됨)
tokenized_dataset = tokenized_dataset.remove_columns(["review"])

# PyTorch Tensor 형식으로 세팅
tokenized_dataset.set_format("torch")
print("✅ 토크나이징 및 텐서 변환 완료")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

✅ 토크나이징 및 텐서 변환 완료


In [9]:
# 6. Train-Test Split (80% 훈련, 20% 테스트)

split_datasets = tokenized_dataset["train"].train_test_split(test_size=0.2)
tokenized_dataset = split_datasets  # 업데이트

print("✅ Train/Test 분할 완료")
print("📊 Train 데이터 개수:", len(tokenized_dataset["train"]))
print("📊 Test 데이터 개수:", len(tokenized_dataset["test"]))


✅ Train/Test 분할 완료
📊 Train 데이터 개수: 80
📊 Test 데이터 개수: 20


In [10]:

# 7. KoBERT 모델 로드 (단일 라벨 분류)
# 예: 라벨 범위가 0~6 총 7개면 num_labels=7
model = AutoModelForSequenceClassification.from_pretrained(
    "skt/kobert-base-v1",
    num_labels=7
)
print("✅ KoBERT 모델 로드 완료")

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ KoBERT 모델 로드 완료


In [11]:
# 라벨이 float 또는 문자열이면 변환
def cast_to_int(example):
    example["label"] = int(example["label"])  # 혹은 np.int64
    return example

tokenized_dataset = tokenized_dataset.map(cast_to_int)


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [12]:
print(tokenized_dataset)


DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 80
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 20
    })
})


In [13]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# 8. Trainer 설정 (배치 크기 조정 및 최적화)
training_args = TrainingArguments(
    output_dir='./results',       # 체크포인트 및 학습 로그 저장 폴더
    num_train_epochs=50,          # 학습 Epoch 수
    per_device_train_batch_size=4,  # ✅ GPU 메모리 최적화 (256 → 4로 조정)
    per_device_eval_batch_size=4,
    save_total_limit=2,           # 체크포인트 저장 개수 제한 (최신 2개만 유지)
    eval_strategy="epoch",        # ✅ 매 epoch마다 검증 수행
    logging_strategy="steps",     # ✅ Training Loss가 로그에 출력되도록 설정
    logging_steps=10,             # ✅ 10 스텝마다 Training Loss 출력
    save_strategy="epoch",        # ✅ Epoch 단위로 체크포인트 저장
    logging_dir='./logs',         # 로그 저장 경로
    save_steps=500,               # 체크포인트 저장 빈도
    report_to="none"              # ✅ W&B 비활성화 (필요하면 "wandb"로 변경 가능)
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

# 🚀 학습 시작
trainer.train()

# ✅ Training Loss 로그 확인
for log in trainer.state.log_history:
    print(log)


Epoch,Training Loss,Validation Loss
1,1.967600,1.969357
2,1.933400,2.014615
3,1.919600,2.030240
4,1.937500,2.005203
5,1.976000,2.064510
6,1.990900,2.037199
7,1.948400,2.002484
8,1.993900,2.016380
9,1.951500,2.027901
10,1.959900,2.058902


{'loss': 2.0387, 'grad_norm': 6.163048267364502, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.5, 'step': 10}
{'loss': 1.9676, 'grad_norm': 7.107999324798584, 'learning_rate': 4.9e-05, 'epoch': 1.0, 'step': 20}
{'eval_loss': 1.9693565368652344, 'eval_runtime': 0.2269, 'eval_samples_per_second': 88.139, 'eval_steps_per_second': 22.035, 'epoch': 1.0, 'step': 20}
{'loss': 1.9812, 'grad_norm': 9.75029182434082, 'learning_rate': 4.85e-05, 'epoch': 1.5, 'step': 30}
{'loss': 1.9334, 'grad_norm': 11.755388259887695, 'learning_rate': 4.8e-05, 'epoch': 2.0, 'step': 40}
{'eval_loss': 2.014615297317505, 'eval_runtime': 0.4222, 'eval_samples_per_second': 47.375, 'eval_steps_per_second': 11.844, 'epoch': 2.0, 'step': 40}
{'loss': 1.9059, 'grad_norm': 9.403446197509766, 'learning_rate': 4.75e-05, 'epoch': 2.5, 'step': 50}
{'loss': 1.9196, 'grad_norm': 4.977385520935059, 'learning_rate': 4.7e-05, 'epoch': 3.0, 'step': 60}
{'eval_loss': 2.030240297317505, 'eval_runtime': 0.2899, 'eval_samples_per

In [14]:
# 9. 모델 학습

print("🚀 학습 시작!")
trainer.train()
print("🚀 학습 완료!")


🚀 학습 시작!


Epoch,Training Loss,Validation Loss
1,1.959900,2.006122
2,1.904400,2.022898
3,1.920200,2.036061
4,1.939100,2.020618
5,1.992300,2.052794
6,1.981200,2.046953
7,1.953300,2.011461
8,1.929800,2.030479
9,1.883100,2.044933
10,1.965200,2.060354


🚀 학습 완료!


In [15]:
# 10. 최종 평가 (Trainer 사용)

print("🔍 테스트 세트 평가 시작...")
predictions = trainer.predict(tokenized_dataset["test"])

# 예측 로짓에서 argmax
preds = torch.argmax(torch.tensor(predictions.predictions), dim=1)

# 실제 라벨
labels = tokenized_dataset["test"]["label"][:]

# 정확도 계산
acc = accuracy_score(labels, preds.tolist())
print(f"🎯 최종 정확도: {acc:.4f}")

🔍 테스트 세트 평가 시작...


🎯 최종 정확도: 0.1000


In [16]:
# 7가지 범주 (0~6)를 한글로 매핑
label_to_category = {
    0: "경치",         # View
    1: "청결도",       # Cleanliness
    2: "호스트서비스", # Host_Service
    3: "편안함",       # Comfort
    4: "접근성",       # Accessibility
    5: "시설",         # Facilities
    6: "반려동물가능"  # Pet_Friendly
}


In [17]:
import torch

def predict_category(model, tokenizer, text):
    # 1) 토크나이징 (token_type_ids 생성하지 않음)
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128,
        return_token_type_ids=False  # ✅ token_type_ids를 생성하지 않음
    )

    # ✅ 모든 입력 데이터를 GPU로 이동
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # 2) 모델 예측
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class = torch.argmax(outputs.logits, dim=1).item()

    return predicted_class

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict_category(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)

    # ✅ 모든 입력 텐서를 GPU로 이동
    for key in inputs.keys():
        inputs[key] = inputs[key].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class = torch.argmax(outputs.logits, dim=1).item()

    return predicted_class


In [19]:
if "token_type_ids" in inputs:
    del inputs["token_type_ids"]


NameError: name 'inputs' is not defined

In [20]:
# ✅ 모델에 입력할 예시 문장
example_review = "숙소에서 보는 경치가 정말 최고였어요!"

# ✅ 토크나이징 후 입력값 확인
inputs = tokenizer(
    example_review,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=128
)

# ✅ 입력값 출력
print("Input IDs:", inputs["input_ids"])
print("Token Type IDs:", inputs["token_type_ids"])
print("Max token_type_id:", max(inputs["token_type_ids"][0]))  # ✅ 가장 큰 token_type_id 값 확인


Input IDs: tensor([[  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1, 517,   0, 491,   0, 517,   0, 517, 490,   0, 494, 490,
           0, 517,   0, 517,   0, 490,   0, 491,   0, 491,   0, 491,   0,   5,
           0,   0]])
Token Type IDs: tensor([[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

In [21]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# 예시 리뷰
example_review = "숙소에서 보는 경치가 정말 최고였어요!"

# 예측 라벨 (정수)
pred_label = predict_category(model, tokenizer, example_review)

# 라벨 → 한글 카테고리 명
category_name = label_to_category[pred_label]

print(f"리뷰 문장: {example_review}")
print(f"예측 라벨 번호: {pred_label}")
print(f"예측된 카테고리: {category_name}")


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
